In [1]:
# coding: utf-8

import socket

simulator_name = 'spiNNaker'

#exec("from pyNN.%s import *" % simulator_name)
from spynnaker.pyNN import *
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility import Timer
import numpy as np

timer = Timer()

2017-06-22 12:32:49 INFO: Read config files: /home/nicolas/.spynnaker.cfg, /home/nicolas/master_thesis/spinnaker_lsm/spinnaker_virtenv/venv/local/lib/python2.7/site-packages/spynnaker/spynnaker.cfg


In [2]:
queens_positions = np.array([0,0,0,0,0,0,0,0])
exc_syn = p.StaticSynapse(delay=1.0, weight=8.0) #6.0
exc_connector = p.OneToOneConnector()
inh_syn = p.StaticSynapse(delay=1.0, weight=-1.0) #-0.2
one_conn = p.OneToOneConnector()
all_conn= p.AllToAllConnector(allow_self_connections=False)
self_conn = p.OneToOneConnector()


NameError: name 'p' is not defined

In [ ]:
cell_type=p.IF_cond_exp()
#cell_type=p.Izhikevich()
test=p.Population(1, cell_type, label="excitatory")
print test.celltype
population_array = np.array([0,0,0,0,0,0,0,0,0], dtype=object)
input_array = np.array([0,0,0,0,0,0,0,0,0], dtype=object)
poisson_input = p.SpikeSourcePoisson(rate=50)
connections={}
for i in np.arange(8):
        population_array[i] = p.Population(8, cell_type, label="excitatory")
        # connect all neurons of the row with each other inhibitory
        connections['row %s' %i]=p.Projection(population_array[i],population_array[i], all_conn, synapse_type=inh_syn)

        # self-connections should be excitatory!!!
        connections['row %s self' %i]=p.Projection(population_array[i],population_array[i], self_conn, synapse_type=exc_syn)
        
        input_array[i]=p.Population(8, poisson_input, label='input')
        # connect every neuron with one input poisson train
        connections['inp%s' %i]=p.Projection(input_array[i],population_array[i],exc_connector,synapse_type=exc_syn)
print population_array[1].celltype
test_input=p.Population(1, poisson_input, label='input')

# add column-wise inhibitory connections
for i in np.arange(8):
    for j in np.arange(8):
        if i!=j:
            connections['column%s' %i]=p.Projection(population_array[i],population_array[j],one_conn,synapse_type=inh_syn)



In [ ]:
population_array[0].record(['spikes'])
population_array[1].record(['spikes'])


global sim_time
sim_time=40
p.run(sim_time)
row_1_data=population_array[0].get_data()
row_2_data=population_array[1].get_data()

In [ ]:
fig_settings = {
    'lines.linewidth': 0.5,
    'axes.linewidth': 0.5,
    'axes.labelsize': 'small',
    'legend.fontsize': 'small',
    'font.size': 8
}
plt.rcParams.update(fig_settings)
plt.figure(1, figsize=(6,8))
def plot_spiketrains(segment,y_label):
    i=0
    for spiketrain in segment.spiketrains:
        neuron_id =spiketrain.annotations['source_id']
        if i==0:
            first_neuron_id=neuron_id
        i+=1
        y = np.ones_like(spiketrain) * neuron_id
        plt.plot(spiketrain, y, '.')
        
        #print spiketrain
        plt.ylabel(y_label)
        plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.axis([0, sim_time, first_neuron_id-1, neuron_id+1])
        
def plot_signal(signal, index, colour='b'):
    label = "Neuron %d" % signal.annotations['source_ids'][index]
    plt.plot(signal.times, signal[:, index], colour, label=label)
    plt.ylabel("%s (%s)" % (signal.name, signal.units._dimensionality.string))
    plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.legend()

In [ ]:
plt.subplot(2, 1, 1)
plot_spiketrains(row_1_data.segments[0],'row_1')
plt.subplot(2,1,2)
plot_spiketrains(row_2_data.segments[0],'row_2')


plt.xlabel("time (ms)")
plt.setp(plt.gca().get_xticklabels(), visible=True)
